In [ ]:
import pandas as pd
import statistics
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
indices = pd.read_csv("./data/indices.csv",index_col='Dates')
#indices = indices['NiKKEI']

In [ ]:
log_returns = np.log(indices.pct_change() + 1)

In [ ]:
#log_returns=log_returns.to_frame()
log_returns=log_returns.dropna()
#log_returns=log_returns.loc[(log_returns!=0).any(axis=1)]
log_returns

In [ ]:
def residual_term_quantile(window, q=0.05):
 return(window).quantile(q)

In [ ]:
historic_var05_7090 = log_returns.rolling(7090).apply(residual_term_quantile, args=(0.05, )).shift(1).dropna()
historic_var01_7090 = log_returns.rolling(7090).apply(residual_term_quantile, args=(0.01, )).shift(1).dropna()

In [ ]:

historic_var01_7090

In [ ]:
historic_var05_7090

In [ ]:
his_sim=historic_var01_7090.mean()
his_sim

In [ ]:
successes = (log_returns[7090:] < historic_var05_7090).sum()
n = len(log_returns[7090:])
print(successes / n)
print('=======')
print('pvalues:')
print(stats.binom_test(successes.iloc[0], n=n, p=0.01))
print(stats.binom_test(successes.iloc[1], n=n, p=0.01))
print(stats.binom_test(successes.iloc[2], n=n, p=0.01))

In [ ]:
successes

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(log_returns['NiKKEI'][7090:].values, label='NIKKEI 225 Daily returns')
plt.plot(historic_var01_7090['NiKKEI'].values, label='99% VaR')
plt.plot(historic_var05_7090['NiKKEI'].values, label='95% VaR')
#plt.title('95% VaR vs 99% VaR')
plt.ylabel('')
plt.xlabel('')
plt.legend(loc='lower left')
plt.show()

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(log_returns['FTSE100'][7090:].values, label='FTSE Daily returns')
plt.plot(historic_var01_7090['FTSE100'].values, label='99% VaR')
plt.plot(historic_var05_7090['FTSE100'].values, label='95% VaR')
#plt.title('95% VaR vs 99% VaR')
plt.ylabel('')
plt.xlabel('')
plt.legend(loc='lower left')
plt.show()

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(log_returns['S&P500'][7090:].values, label='S&P 500 Daily returns')
plt.plot(historic_var01_7090['S&P500'].values, label='99% VaR')
plt.plot(historic_var05_7090['S&P500'].values, label='95% VaR')
#plt.title('95% VaR vs 99% VaR')
plt.ylabel('')
plt.xlabel('')
plt.legend(loc='lower left')
plt.show()

In [ ]:
from arch import arch_model

In [ ]:
from arch.univariate import ZeroMean, ConstantMean, ARX, GARCH, StudentsT, HARX, SkewStudent
from arch.__future__ import reindexing

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
def rolling_gjrgarch(window, quantile=0.01):
    scale = 100
    model = arch_model(scale * window, vol='GARCH', p=1, q=1, dist='t') # o=1 leads to GRJGARCH model
    #model = ARX(scale * window, lags=1)
    #model = HARX(scale * window, lags=1)
    #model = ZeroMean(scale * window)
    #model.volatility = GARCH(1,1)
    # model.distribution = StudentsT()
    #model.distribution = SkewStudent()
    res = model.fit()
    #print(res.summary())
    forecasts = res.forecast(horizon=1)
    cond_mean = forecasts.mean.iloc[-1]
    print('cond_mean', cond_mean)
    cond_var = forecasts.variance.iloc[-1]
    print('cond_var', cond_var)
    q = model.distribution.ppf(quantile, parameters=res.params[-1:])
    value_at_risk = - cond_mean.values - np.sqrt(cond_var) * q
    return - value_at_risk / scale

In [ ]:
gjrgarch_i_01 = pd.DataFrame({
    k: log_returns[k].rolling(7090).apply(rolling_gjrgarch, args=(0.01,)).shift(1)
    for k in log_returns.columns
})

In [ ]:
gjrgarch_i_05 = pd.DataFrame({
    k: log_returns[k].rolling(7090).apply(rolling_gjrgarch, args=(0.05,)).shift(1)
    for k in log_returns.columns
})

In [ ]:
gjrgarch_i_01[7090:]

In [ ]:
successes = (log_returns[7090:] < gjrgarch_i_05[7090:]).sum()
n = len(log_returns[7090:])
print(successes / n)
print('=======')
print('pvalues:')
print(stats.binom_test(successes.iloc[0], n=n, p=0.01))
print(stats.binom_test(successes.iloc[1], n=n, p=0.01))
print(stats.binom_test(successes.iloc[2], n=n, p=0.01))

In [ ]:
successes

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(log_returns['NiKKEI'][7090:].values, label='NIKKEI 225 Daily returns')
plt.plot(gjrgarch_i_01[7090:]['NiKKEI'].values, label='99% VaR')
plt.plot(gjrgarch_i_05[7090:]['NiKKEI'].values, label='95% VaR')
#plt.title('95% VaR vs 99% VaR')
plt.ylabel('')
plt.xlabel('')
plt.legend(loc='upper center')
plt.show()

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(log_returns['FTSE100'][7090:].values, label='FTSE Daily returns')
plt.plot(gjrgarch_i_01[7090:]['FTSE100'].values, label='99% VaR')
plt.plot(gjrgarch_i_05[7090:]['FTSE100'].values, label='95% VaR')
#plt.title('95% VaR vs 99% VaR')
plt.ylabel('')
plt.xlabel('')
plt.legend(loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(log_returns['S&P500'][7090:].values, label='S&P 500 Daily returns')
plt.plot(gjrgarch_i_01[7090:]['S&P500'].values, label='99% VaR')
plt.plot(gjrgarch_i_05[7090:]['S&P500'].values, label='95% VaR')
#plt.title('95% VaR vs 99% VaR')
plt.ylabel('')
plt.xlabel('')
plt.legend(loc='upper left')
plt.show()

In [ ]:
KK=(0.99**1363)*(0.01**23)
KK

In [ ]:
JJ= ((1-(23/1386))**1363)*((23/1386)**23)
JJ

In [ ]:
KK/JJ

In [ ]:
-8.755764454947405*-2